In [460]:
"""
Истпытательный скрипт по Учебным дисциплинам
"""

'\nИстпытательный скрипт по Учебным дисциплинам\n'

In [461]:
import pandas as pd
import numpy as np
import openpyxl
from docxtpl import DocxTemplate
import string
import time
import re

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.expand_frame_repr', False)  # Не переносить строки
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=FutureWarning, module='openpyxl')

In [462]:
df_plan_ud = pd.read_excel('data/Автозаполнение РП.xlsx',sheet_name='План УД',usecols='A:F')
df_plan_ud.dropna(inplace=True, thresh=1)  # удаляем пустые строки

In [463]:
df_plan_ud.columns = ['Раздел','Тема','Количество_часов','Практика','Вид_занятия','СРС']

In [464]:
df_plan_ud['Раздел'].fillna('Пусто',inplace=True)

In [465]:
borders=df_plan_ud[df_plan_ud['Раздел'].str.contains('семестр')].index # получаем индексы строк где есть слово семестр

In [466]:
part_df = [] # список для хранения кусков датафрейма
previos_border = -1
# делим датафрем по границам
for value_border in borders:
    part = df_plan_ud.iloc[previos_border:value_border]
    part_df.append(part)
    previos_border = value_border


In [467]:
# добавляем последнюю часть
last_part = df_plan_ud.iloc[borders[-1]:]
part_df.append(last_part)

In [468]:
part_df.pop(0) # удаляем нулевой элемент так как он пустой

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС


In [469]:
main_df = pd.DataFrame(columns=['Раздел','Тема','Количество_часов','Практика','Вид_занятия','СРС']) #  создаем базовый датафрейм

In [470]:
lst_type_lesson= ['урок','практическое занятие','лабораторное занятие','курсовая работа (КП)'] # список типов занятий
for df in part_df:
    dct_sum_result = {key:0 for key in lst_type_lesson} # создаем словарь для подсчета значений
    for type_lesson in lst_type_lesson:
        _df = df[df['Вид_занятия'] == type_lesson] # фильтруем датафрейм
        _df['Количество_часов'].fillna(0,inplace=True)
        _df['Количество_часов'] = _df['Количество_часов'].astype(int)
        dct_sum_result[type_lesson] = _df['Количество_часов'].sum()
    # создаем строку с описанием
    margint_text = 'Итого часов за семестр:\nиз них\nтеория\nпрактические занятия\nлабораторные занятия\nкурсовая работа (КП)'

    all_hours = sum(dct_sum_result.values()) # общая сумма часов

    theory_hours = dct_sum_result['урок'] # часы теории
    praktice_hours = dct_sum_result['практическое занятие'] # часы практики
    lab_hours = dct_sum_result['лабораторное занятие'] # часы лабораторных
    kurs_hours = dct_sum_result['курсовая работа (КП)'] # часы курсовых

    value_text = f'{all_hours}\n""\n{theory_hours}\n{praktice_hours}\n{lab_hours}\n{kurs_hours}' # строка со значениями
    #itog_row = {'Тема':margint_text,'Количество_часов':value_text} # создаем строку
    temp_df = pd.DataFrame([{'Тема':margint_text,'Количество_часов':value_text}])
    df = pd.concat([df,temp_df],ignore_index=True) # добаляем итоговую строку
    main_df = pd.concat([main_df,df],ignore_index=True) # добавляем в основной датафрейм
        
        
        


In [471]:
main_df.insert(0,'№_занятия',np.nan) # добавляем колонку с номерами занятий

In [472]:
main_df['Тема'] =main_df['Тема'].fillna('Пусто') # заменяем наны на пусто

In [473]:
count = 0 # счетчик
for idx,row in enumerate(main_df.itertuples()):
    if (row[3] == 'Пусто') | ('Итого часов' in row[3]):
        main_df.iloc[idx,0] = ''
    else:
        count += 1
        main_df.iloc[idx,0] = count
        

In [474]:
# очищаем от пустых символов и строки Пусто
main_df['Раздел'] = main_df['Раздел'].fillna('Пусто')
main_df['Тема'] = main_df['Тема'].replace('Пусто','')
main_df['Раздел'] = main_df['Раздел'].replace('Пусто','')

In [475]:
main_df

,№_занятия,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС
0,,1 курс \n1 семестр,,NaN,NaN,NaN,NaN
1,1,,Грамматически основы построения предложений,2.0,NaN,урок,NaN
2,2,,История железнодорожного транспорта.,2.0,NaN,практическое занятие,NaN
3,3,,История железнодорожного транспорта.,2.0,NaN,практическое занятие,NaN
4,4,,Первая железная дорога в России.,2.0,NaN,практическое занятие,NaN
...,...,...,...,...,...,...,...
68,62,,Правила перевозки домашних животных,2.0,NaN,практическое занятие,NaN
69,63,,"Вещества и предметы, запрещенные к провозу на ...",2.0,NaN,практическое занятие,NaN
70,64,,Итоговая контрольная работа,2.0,NaN,практическое занятие,NaN
71,65,,Дифференцированный зачет за 4 семестр,2.0,NaN,практическое занятие,NaN


In [476]:
main_df['Тема'] = main_df['Раздел'] + main_df['Тема']

In [477]:
main_df.drop(columns=['Раздел'],inplace=True)

In [478]:
main_df.to_excel('fd.xlsx',index=False)